**OBLIGATORIO FINAL G448 - Data Wrangling**
<br>**INTEGRANTES:**
<br>
+ Theo Pereira, 5703834-3, theopereira2001@gmail.com
+ Carlos Píriz, 4298784-4, piriz.carlos@gmail.com
+ Gabriel	Better,3138871-2, gabrielbetter@gmail.com

In [202]:
### Importacion de librerias ###

# Sistema
import os

# Tratamiento de datos
import pandas as pd
import numpy as np
import datetime as dt
import math

# Visualizacion 
import seaborn as sns
import matplotlib as mpl
from matplotlib import ticker
from matplotlib.ticker import FuncFormatter
from babel.numbers import format_currency
import matplotlib.pyplot as plt
%matplotlib inline

**CONSIGNAS**
<br>
El objetivo general es realizar un análisis exploratorio de los anuncios inmobiliarios de
algunas provincias de la Patagonia Argentina reportados por el portal Airbnb.
<br>
Los datos están conformados por cuatro archivos csv:
+ Neuquen.csv
+ Rio_Negro.csv
+ Chubut.csv
+ Tierra_del_Fuego.csv

**1. Carga de datos y armado del dataset**

In [203]:
# Carga de archivos de datos de chubut
df_chubut = pd.read_csv('https://raw.githubusercontent.com/carlos1680/CPEDWObligatorio/main/Chubut.csv')
df_chubut["provincia"]="CHUBUT"
df_chubut.sample(10)

,id_alojamiento,name,category,rating,city,latitude,longitude,characteristics,check_in,check_out,price_discounted,price_original,price_qualifier,cant_resultados,fecha_scrapping,fecha_cambio_bna,cambio_bna,provincia
221,54362650,"""Ty Bychan"" \nLa Casita. Entre raíces galesas.",entire_home,"5,0 (3)",Gaiman,-43.286520,-65.494660,"['3 camas', '2 dormitorios', '1 baño', '1 cama...",2023-07-01,2023-07-06,NaN,$32 USD,noche,1000,2023-06-24,2023-05-01,239.919355,CHUBUT
118,887853891920643116,"Luma, tu lugar feliz en el sur",entire_home,Nuevo,Esquel,-42.917491,-71.294685,"['1 cama', '1 dormitorio', '1 baño']",2023-07-01,2023-07-06,$34 USD,$42 USD,noche,1000,2023-06-24,2023-05-01,239.919355,CHUBUT
71,586216477715336949,Encantador dpto. con vista al mar en Puerto Ma...,entire_home,"5,0 (11)",Puerto Madryn,-42.771633,-65.029068,"['3 camas', '2 dormitorios', '2,5 baños', 'Cal...",2023-07-06,2023-07-11,NaN,$131 USD,noche,1000,2023-06-24,2023-05-01,239.919355,CHUBUT
193,839910728991938719,"""El Rancho"" Sentirte en casa",entire_home,NaN,Lago Puelo,-42.034017,-71.545491,"['3 camas', '1 dormitorio', '1 baño']",2023-07-01,2023-07-06,NaN,$47 USD,noche,1000,2023-06-24,2023-05-01,239.919355,CHUBUT
127,21925703,Hermoso Duplex con Patio y Parrilla. (4 pax).,entire_home,"4,81 (74)",Puerto Madryn,-42.787700,-65.023850,"['3 camas', '2 dormitorios', '1 baño', '1 cama...",2023-07-01,2023-07-06,NaN,$57 USD,noche,1000,2023-06-24,2023-05-01,239.919355,CHUBUT
15,767503846951993933,Brisa de Mar - Puerto Madryn,entire_home,"4,95 (22)",Puerto Madryn,-42.771634,-65.029071,"['3 camas', '2 dormitorios', '2,5 baños', 'Cal...",2023-08-07,2023-08-12,$34 USD,$48 USD,noche,1000,2023-06-24,2023-05-01,239.919355,CHUBUT
183,899344747833979419,Mono ambiente en Trelew,entire_home,Nuevo,Trelew,-43.268183,-65.311408,"['3 camas', 'Monoambiente', '1 baño', '1 cama ...",2023-07-01,2023-07-06,$24 USD,$30 USD,noche,1000,2023-06-24,2023-05-01,239.919355,CHUBUT
174,15919724,"Duplex luminoso, 2 cuadras de la playa",entire_home,"4,8 (31)",Puerto Madryn,-42.783870,-65.019680,"['2 camas', '1 dormitorio', '1 baño', '1 cama ...",2023-07-01,2023-07-06,NaN,$40 USD,noche,1000,2023-06-24,2023-05-01,239.919355,CHUBUT
219,10192112,Las Nativas | Cabaña Vista al cerro Ideal pare...,entire_home,"4,81 (69)",Lago Puelo,-42.029880,-71.584660,"['2 camas', '1 dormitorio', '1 baño', '1 cama ...",2023-07-01,2023-07-06,NaN,$48 USD,noche,1000,2023-06-24,2023-05-01,239.919355,CHUBUT
155,726181842782357680,Departamento en el centro con estacionamiento,entire_home,"4,85 (13)",Puerto Madryn,-42.770075,-65.033935,"['3 camas', '2 dormitorios', '1 baño', '1 cama...",2023-07-01,2023-07-06,NaN,$75 USD,noche,1000,2023-06-24,2023-05-01,239.919355,CHUBUT


In [204]:
# Carga de archivos de datos de Neuquen
df_neuquen = pd.read_csv('https://raw.githubusercontent.com/carlos1680/CPEDWObligatorio/main/Neuquen.csv')

df_neuquen["provincia"] = "NEUQUEN"

df_neuquen.sample(10)

,id_alojamiento,name,category,rating,city,latitude,longitude,characteristics,check_in,check_out,price_discounted,price_original,price_qualifier,cant_resultados,fecha_scrapping,fecha_cambio_bna,cambio_bna,provincia
192,15774741,Casa muy cómoda cerca del río y paseos,entire_home,NaN,Neuquen,-38.918310,-68.055250,"['No hay evaluaciones', '2 camas', '2 dormitor...",2023-07-01,2023-07-06,NaN,$75 USD,noche,298,2023-06-24,2023-05-01,239.919355,NEUQUEN
183,848876725737805421,Departamento céntrico JUANA,entire_home,NaN,Neuquén,-38.950567,-68.068210,"['No hay evaluaciones', '1 cama', '1 dormitori...",2023-07-01,2023-07-06,NaN,$40 USD,noche,298,2023-06-24,2023-05-01,239.919355,NEUQUEN
33,54111994,Hermosa cabaña para descansar Plottier. Neuquen,entire_home,"4,83 (30)",Neuquen,-38.965480,-68.253840,"['5 camas', '2 dormitorios', '1 baño', '1 cama...",2023-07-01,2023-07-06,NaN,$50 USD,noche,298,2023-06-24,2023-05-01,239.919355,NEUQUEN
256,36144936,Dpto nuevo con aire en una hermosa zona.,entire_home,"4,89 (19)",Neuquén,-38.947090,-68.048020,"['1 cama', '1 dormitorio', '1 baño', '1 cama d...",2023-08-01,2023-08-07,NaN,$48 USD,noche,298,2023-06-24,2023-05-01,239.919355,NEUQUEN
169,51735487,Departamento con balcón en complejo residencial,entire_home,NaN,Cipolletti,-38.928150,-67.999340,"['1 cama', '1 dormitorio', '1 baño', 'Wifi']",2023-07-01,2023-07-06,NaN,$51 USD,noche,298,2023-06-24,2023-05-01,239.919355,NEUQUEN
37,14910454,Apartamento Muy Còmodo y Espacioso ( Patagonia ),entire_home,"4,92 (92)",Neuquén,-38.951000,-68.104510,"['4 camas', '2 dormitorios', '1 baño', '1 cama...",2023-07-01,2023-07-06,NaN,$32 USD,noche,298,2023-06-24,2023-05-01,239.919355,NEUQUEN
60,8378298,Luminosa cabaña en amplio parque,entire_home,"4,87 (70)",Plottier,-38.959160,-68.227750,"['3 camas', '1 dormitorio', '1 baño', '1 cama ...",2023-07-01,2023-07-07,NaN,$36 USD,noche,298,2023-06-24,2023-05-01,239.919355,NEUQUEN
181,822601586110686138,Casa en Puerto Manzano,entire_home,NaN,Neuquén,-38.954100,-68.095960,"['No hay evaluaciones', '7 camas', '3 dormitor...",2023-07-01,2023-07-06,NaN,$151 USD,noche,298,2023-06-24,2023-05-01,239.919355,NEUQUEN
223,54181126,Hermoso estudio centro de Cipo! Amueblados Com...,entire_home,NaN,Cipolletti,-38.939580,-68.000750,"['3 camas', 'Monoambiente', '1 baño', '2 camas...",2023-07-01,2023-07-06,NaN,$50 USD,noche,298,2023-06-24,2023-05-01,239.919355,NEUQUEN
25,717469779653882476,Luminoso y cálido monoambiente en Neuquén,entire_home,"4,96 (23)",Neuquén,-38.953987,-68.047692,"['1 cama', '1 dormitorio', '1 baño', 'Wifi', '...",2023-07-01,2023-07-06,NaN,$30 USD,noche,298,2023-06-24,2023-05-01,239.919355,NEUQUEN


In [205]:
# Carga de archivos de datos de Rio Negro
df_rio_negro = pd.read_csv('https://raw.githubusercontent.com/carlos1680/CPEDWObligatorio/main/Rio_Negro.csv')

df_rio_negro["provincia"] = "RIO NEGRO"

df_rio_negro.sample(10)

,id_alojamiento,name,category,rating,city,latitude,longitude,characteristics,check_in,check_out,price_discounted,price_original,price_qualifier,cant_resultados,fecha_scrapping,fecha_cambio_bna,cambio_bna,provincia
15,53403878,Peta Lodge Patagonia I,entire_home,"4,95 (93)",San Carlos de Bariloche,-41.117900,-71.382700,"['2 camas', '1 dormitorio', '1 baño', 'Estacio...",2023-08-17,2023-08-23,NaN,$138 USD,noche,1000,2023-06-24,2023-05-01,239.919355,RIO NEGRO
149,550807374525703424,Acogedor estudio en la costa del lago,entire_home,"4,97 (31)",San Carlos de Bariloche,-41.071040,-71.530250,"['1 cama', 'Monoambiente', '1 baño', 'Estacion...",2023-07-01,2023-07-06,NaN,$95 USD,noche,1000,2023-06-24,2023-05-01,239.919355,RIO NEGRO
52,46903374,"""Casita"" a estrenar, 21 m2 de luz",entire_home,"4,6 (40)",San Carlos de Bariloche,-41.090300,-71.452320,"['1 cama', 'Monoambiente', '1 cama doble', 'Es...",2023-07-16,2023-07-21,NaN,$40 USD,noche,1000,2023-06-24,2023-05-01,239.919355,RIO NEGRO
22,793489760148856223,Ruquita Studio Schuss Arg41,entire_home,"4,92 (12)",San Carlos de Bariloche,-41.121500,-71.372480,"['1 cama', 'Monoambiente', '1 baño', 'Estacion...",2023-07-21,2023-07-26,NaN,$59 USD,noche,1000,2023-06-24,2023-05-01,239.919355,RIO NEGRO
208,899090354979371279,Costa Nahuel Apartments,entire_home,Nuevo,Dina Huapi,-41.078048,-71.179887,"['3 camas', '2 dormitorios', '1 baño', 'Estaci...",2023-07-07,2023-07-12,$80 USD,$100 USD,noche,1000,2023-06-24,2023-05-01,239.919355,RIO NEGRO
70,699829944583701481,Cabaña en Villa la Angostura,entire_home,"4,89 (9)",Villa la Angostura,-40.762909,-71.628681,"['4 camas', '3 dormitorios', '2 baños', 'Calif...",2023-07-29,2023-08-03,NaN,$250 USD,noche,1000,2023-06-24,2023-05-01,239.919355,RIO NEGRO
95,6060731,Cabaña Americana con pileta climatizada,entire_home,"4,56 (264)",San Carlos de Bariloche,-41.065070,-71.157830,"['4 camas', '2 dormitorios', '1 baño', 'Estaci...",2023-08-05,2023-08-10,NaN,$63 USD,noche,1000,2023-06-24,2023-05-01,239.919355,RIO NEGRO
179,4635954,"ESTUDIO A 20 METROS DEL CIVICO CENTER, VISTA A...",entire_home,"4,68 (175)",San Carlos de Bariloche,-41.133920,-71.310280,"['1 cama', 'Monoambiente', '1 ,5 baños', 'Wifi...",2023-08-03,2023-08-08,NaN,$96 USD,noche,1000,2023-06-24,2023-05-01,239.919355,RIO NEGRO
241,31512214,Hermosa Cabaña en Villa Traful,entire_home,"4,89 (19)",Villa Traful,-40.649690,-71.417250,"['1 cama', '1 dormitorio', '1 baño', '1 cama d...",2023-07-01,2023-07-06,NaN,$65 USD,noche,1000,2023-06-24,2023-05-01,239.919355,RIO NEGRO
122,27109935,Refugio de montaña impecable,entire_home,"4,9 (20)",Villa La Angostura,-40.760500,-71.625410,"['2 camas', '1 dormitorio', '1 baño', '1 cama ...",2023-07-01,2023-07-06,NaN,$105 USD,noche,1000,2023-06-24,2023-05-01,239.919355,RIO NEGRO


In [206]:
# Carga de archivos de datos de Tierra delFuego
df_tierra_del_fuego = pd.read_csv('https://raw.githubusercontent.com/carlos1680/CPEDWObligatorio/main/Tierra_del_Fuego.csv')

df_tierra_del_fuego["provincia"] = "TIERRA DEL FUEGO"

df_tierra_del_fuego.sample(10)

,id_alojamiento,name,category,rating,city,latitude,longitude,characteristics,check_in,check_out,price_discounted,price_original,price_qualifier,cant_resultados,fecha_scrapping,fecha_cambio_bna,cambio_bna,provincia
109,862480988954392344,Habitación doble encantadora,private_room,NaN,Ushuaia,-54.806550,-68.331390,"['1 cama', '1 dormitorio', '2 baños compartido...",2023-07-01,2023-07-08,NaN,$23 USD,noche,1000,2023-06-24,2023-05-01,239.919355,TIERRA DEL FUEGO
148,871110423239218950,En_Ushuaia II,entire_home,"4,86 (7)",Ushuaia,-54.824280,-68.344210,"['1 cama', 'Monoambiente', '1 baño', 'Wifi', '...",2023-07-01,2023-07-06,NaN,$29 USD,noche,1000,2023-06-24,2023-05-01,239.919355,TIERRA DEL FUEGO
54,604348508600425631,Departamentos “Al Sur”,entire_home,"4,96 (23)",Ushuaia,-54.815620,-68.327640,"['2 camas', '1 dormitorio', '1 baño', '1 cama ...",2023-07-11,2023-07-16,$48 USD,$57 USD,noche,1000,2023-06-24,2023-05-01,239.919355,TIERRA DEL FUEGO
94,890677846374836692,Krund Bed and Bikes,entire_home,"5,0 (5)",Ushuaia,-54.802141,-68.317699,"['1 cama', 'Monoambiente', '1 baño', 'Wifi', '...",2023-07-21,2023-07-27,NaN,$47 USD,noche,1000,2023-06-24,2023-05-01,239.919355,TIERRA DEL FUEGO
74,742274618573325014,Habitación en departamento particular en Ushuaia,private_room,"4,89 (18)",Ushuaia,-54.830980,-68.361710,"['2 camas', '1 dormitorio', '1 baño compartido...",2023-07-01,2023-07-06,NaN,$17 USD,noche,1000,2023-06-24,2023-05-01,239.919355,TIERRA DEL FUEGO
257,753735338164221801,Seaport - Austral 402 Como en casa lejos de casa.,entire_home,NaN,Ushuaia,-54.806335,-68.309508,"['2 camas', 'Monoambiente', '1 ,5 baños', 'Wif...",2023-07-01,2023-07-06,NaN,$150 USD,noche,1000,2023-06-24,2023-05-01,239.919355,TIERRA DEL FUEGO
207,831611894265914063,Ushuaia home,private_room,"4,5 (4)",Ushuaia,-54.821110,-68.326990,"['1 cama', '1 dormitorio', '1 baño compartido'...",2023-07-01,2023-07-06,NaN,$28 USD,noche,1000,2023-06-24,2023-05-01,239.919355,TIERRA DEL FUEGO
71,755135392594127489,Encantadoras vistas del canal de beagle - Amun...,entire_home,"5,0 (19)",Ushuaia,-54.810100,-68.319410,"['3 camas', '1 dormitorio', '1 baño', 'Wifi', ...",2023-07-01,2023-07-06,NaN,$85 USD,noche,1000,2023-06-24,2023-05-01,239.919355,TIERRA DEL FUEGO
203,13155307,Increíble vista al Beagle.,entire_home,"4,8 (250)",Ushuaia,-54.809280,-68.319080,"['3 camas', '1 dormitorio', '1 baño', '1 cama ...",2023-08-23,2023-08-28,$61 USD,$77 USD,noche,1000,2023-06-24,2023-05-01,239.919355,TIERRA DEL FUEGO
138,889733753812060778,Ushuaia Mágica - Ush AustraLis,entire_home,Nuevo,Ushuaia,-54.811890,-68.326950,"['2 camas', 'Monoambiente', '1 baño', 'Wifi', ...",2023-07-09,2023-07-14,NaN,$45 USD,noche,1000,2023-06-24,2023-05-01,239.919355,TIERRA DEL FUEGO


In [207]:
# Verificacion de columnas para concatenar
(df_chubut.columns == df_neuquen.columns) & (df_rio_negro.columns == df_tierra_del_fuego.columns) & (df_chubut.columns == df_tierra_del_fuego.columns)

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True])

In [208]:
# Concatenacion de los dataframes separados por provincias en un unico archivo
df_alquileres = pd.concat([df_chubut, df_neuquen, df_rio_negro, df_tierra_del_fuego])
df_alquileres.sample(10)

,id_alojamiento,name,category,rating,city,latitude,longitude,characteristics,check_in,check_out,price_discounted,price_original,price_qualifier,cant_resultados,fecha_scrapping,fecha_cambio_bna,cambio_bna,provincia
153,739302775005502135,Hermoso y moderno departamento! estadías largas,entire_home,NaN,JKB,-38.946970,-68.050060,"['1 cama', '1 dormitorio', '1 baño', 'Wifi']",2023-07-01,2023-07-06,NaN,$44 USD,noche,298,2023-06-24,2023-05-01,239.919355,NEUQUEN
172,690413283545175725,Las Aguilas 3,entire_home,"4,67 (9)",Ushuaia,-54.814922,-68.341647,"['4 camas', '1 dormitorio', '1 baño', 'Wifi', ...",2023-07-08,2023-07-14,NaN,$50 USD,noche,1000,2023-06-24,2023-05-01,239.919355,TIERRA DEL FUEGO
183,899344747833979419,Mono ambiente en Trelew,entire_home,Nuevo,Trelew,-43.268183,-65.311408,"['3 camas', 'Monoambiente', '1 baño', '1 cama ...",2023-07-01,2023-07-06,$24 USD,$30 USD,noche,1000,2023-06-24,2023-05-01,239.919355,CHUBUT
248,14360783,"Casa en Playa unión, ballenas, surf, Patagonia.",entire_home,"4,75 (4)",Playa Union,-43.314420,-65.051780,"['4 camas', '2 dormitorios', '1 ,5 baños', '1 ...",2023-07-01,2023-07-06,NaN,$59 USD,noche,1000,2023-06-24,2023-05-01,239.919355,CHUBUT
194,47975732,"""La Posadita"" Casa de Montaña, Miralejos",entire_home,"5,0 (3)",San Martin de los Andes,-40.163160,-71.283120,"['5 camas', '3 dormitorios', '2 baños', '1 cam...",2023-07-01,2023-07-06,NaN,$250 USD,noche,1000,2023-06-24,2023-05-01,239.919355,RIO NEGRO
245,13941382,Cabañas MARI MARI (PUELO),entire_home,NaN,El Hoyo,-42.057100,-71.541730,"['No hay evaluaciones', '3 camas', '2 dormitor...",2023-07-01,2023-07-06,NaN,$59 USD,noche,1000,2023-06-24,2023-05-01,239.919355,CHUBUT
192,837061192294336671,Loft del Fin del Mundo,entire_home,"5,0 (3)",Ushuaia,-54.831543,-68.353667,"['3 camas', '1 dormitorio', '1 ,5 baños', 'Wif...",2023-07-01,2023-07-06,NaN,$59 USD,noche,1000,2023-06-24,2023-05-01,239.919355,TIERRA DEL FUEGO
147,41389351,alojamiento entero\n4 huespedes,entire_home,NaN,Neuquén,-38.976250,-68.043400,"['No hay evaluaciones', '3 camas', '1 dormitor...",2023-07-01,2023-07-06,NaN,$51 USD,noche,298,2023-06-24,2023-05-01,239.919355,NEUQUEN
140,41688833,Casa con vista y acceso al Lago en El Chocón,entire_home,NaN,Neuquén,-38.951500,-68.058880,"['4 camas', '2 dormitorios', '1 baño', '1 cama...",2023-07-01,2023-07-06,NaN,$59 USD,noche,298,2023-06-24,2023-05-01,239.919355,NEUQUEN
232,30035722,Casa de montaña c/ vista al lago - Arrayanes,entire_home,"4,94 (35)",San Carlos de Bariloche,-41.076040,-71.464680,"['4 camas', '2 dormitorios', '1 baño', '1 cama...",2023-09-09,2023-09-14,NaN,$79 USD,noche,1000,2023-06-24,2023-05-01,239.919355,RIO NEGRO


In [209]:
# Reseteo del index, permitiendo duplicados porque aun no sabemos si son unicos los valores
# df_alquileres.reset_index(names = 'id_alojamiento', drop=True,inplace=True,allow_duplicates= True)
# df_alquileres.reset_index(names = 'id_alojamiento', drop=True,inplace=True,allow_duplicates= False)
df_alquileres.reset_index(drop=True,inplace=True)

*Verificacion de filas de los dataframes*

In [210]:
# Verificacion de la cantidad de filas de cada dataframe
print("La cantidad de filas del dataframe de Chubut es:"+str(df_chubut.shape[0]))
print("La cantidad de filas del dataframe de Neuquen es:"+str(df_neuquen.shape[0]))
print("La cantidad de filas del dataframe de Rio Negro es:"+str(df_rio_negro.shape[0]))
print("La cantidad de filas del dataframe de Tierra del fuego es:" + str(df_tierra_del_fuego.shape[0]))
print("La cantidad de filas del dataframe unificado de alquileres es:" + str(df_alquileres.shape[0]))


La cantidad de filas del dataframe de Chubut es:270
La cantidad de filas del dataframe de Neuquen es:270
La cantidad de filas del dataframe de Rio Negro es:270
La cantidad de filas del dataframe de Tierra del fuego es:270
La cantidad de filas del dataframe unificado de alquileres es:1080


*Verificacion de las columnas de los dataframes*

In [211]:
# Verificacion de la cantidad de columnas de cada dataframe
print("La cantidad de columnas del dataframe de Chubut es:" +
      str(df_chubut.shape[1]))
print("La cantidad de columnas del dataframe de Neuquen es:" +
      str(df_neuquen.shape[1]))
print("La cantidad de columnas del dataframe de Rio Negro es:" +
      str(df_rio_negro.shape[1]))
print("La cantidad de columnas del dataframe de Tierra del fuego es:" +
      str(df_tierra_del_fuego.shape[1]))
print("La cantidad de columnas del dataframe unificado de alquileres es:" +
      str(df_alquileres.shape[1]))

La cantidad de columnas del dataframe de Chubut es:18
La cantidad de columnas del dataframe de Neuquen es:18
La cantidad de columnas del dataframe de Rio Negro es:18
La cantidad de columnas del dataframe de Tierra del fuego es:18
La cantidad de columnas del dataframe unificado de alquileres es:18


*Verificacion general e inicial del dataframe unificado*

In [212]:
# Informacion de las columnas, sus tipos y faltantes
df_alquileres.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1080 entries, 0 to 1079
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id_alojamiento    1080 non-null   int64  
 1   name              1080 non-null   object 
 2   category          1080 non-null   object 
 3   rating            886 non-null    object 
 4   city              1076 non-null   object 
 5   latitude          1080 non-null   float64
 6   longitude         1080 non-null   float64
 7   characteristics   1080 non-null   object 
 8   check_in          1080 non-null   object 
 9   check_out         1080 non-null   object 
 10  price_discounted  201 non-null    object 
 11  price_original    1080 non-null   object 
 12  price_qualifier   1080 non-null   object 
 13  cant_resultados   1080 non-null   int64  
 14  fecha_scrapping   1080 non-null   object 
 15  fecha_cambio_bna  1080 non-null   object 
 16  cambio_bna        1080 non-null   float64


In [213]:
# Informacion de las columnas, sus tipos y faltantes
df_alquileres.describe()

,id_alojamiento,latitude,longitude,cant_resultados,cambio_bna
count,1.080000e+03,1080.000000,1080.000000,1080.000000,1080.000000
mean,3.685341e+17,-44.426201,-68.723924,824.500000,239.919355
std,3.903051e+17,6.164758,2.159715,304.115744,0.000000
min,4.468310e+05,-54.844914,-71.688000,298.000000,239.919355
25%,3.529220e+07,-47.854503,-71.310280,824.500000,239.919355
50%,5.408344e+07,-42.003780,-68.305610,1000.000000,239.919355
75%,7.872495e+17,-39.609811,-68.049328,1000.000000,239.919355
max,9.198428e+17,-38.874370,-62.920960,1000.000000,239.919355


In [214]:
# Informacion de las columnas, sus tipos y faltantes
df_alquileres.duplicated(keep="first").sum()

51

In [215]:
# Informacion de las columnas, sus tipos y faltantes
df_alquileres.isna().sum()

id_alojamiento        0
name                  0
category              0
rating              194
city                  4
latitude              0
longitude             0
characteristics       0
check_in              0
check_out             0
price_discounted    879
price_original        0
price_qualifier       0
cant_resultados       0
fecha_scrapping       0
fecha_cambio_bna      0
cambio_bna            0
provincia             0
dtype: int64

*Conclusiones iniciales*
<br>
+ El dataframe unificado, coincide en cantidad de filas y columnas correspondientes a los dataframes individuales
+ Debemos de corregir algunos tipos de datos de columnas
+ Tenemos algunas columnas para reescalar
+ Tenemos filas duplicadas para revisar
+ Tenemos valores nulos para revisar, en principio para las columnas rating y price_discounted

**2. Análisis de duplicados**

*Analisis y tratamiento de duplicados global*

In [216]:
# Verifico los duplicados, que coinciden en toda la fila
df_alquileres.duplicated(keep="first").sum()

51

In [217]:
#  Listo los valores duplicados
df_alquileres[df_alquileres.duplicated(keep="first")]

,id_alojamiento,name,category,rating,city,latitude,longitude,characteristics,check_in,check_out,price_discounted,price_original,price_qualifier,cant_resultados,fecha_scrapping,fecha_cambio_bna,cambio_bna,provincia
54,40755147,La Amarilla Casa de Huéspedes (con cocina) Rut...,entire_home,"4,93 (27)",Epuyén,-42.227509,-71.362289,"['2 camas', '1 dormitorio', '1 ,5 baños', '1 c...",2023-07-01,2023-07-06,NaN,$45 USD,noche,1000,2023-06-24,2023-05-01,239.919355,CHUBUT
72,586216477715336949,Encantador dpto. con vista al mar en Puerto Ma...,entire_home,"5,0 (11)",Puerto Madryn,-42.771633,-65.029068,"['3 camas', '2 dormitorios', '2,5 baños', 'Cal...",2023-07-06,2023-07-11,NaN,$131 USD,noche,1000,2023-06-24,2023-05-01,239.919355,CHUBUT
90,37836757,Rincón del Sol,entire_home,"4,4 (5)",Puerto Madryn,-42.751650,-65.053520,"['2 camas', '1 dormitorio', '1 baño', '1 cama ...",2023-07-01,2023-07-06,NaN,$18 USD,noche,1000,2023-06-24,2023-05-01,239.919355,CHUBUT
113,736253141994173324,Refugio Ciprés con tina de agua caliente,entire_home,"5,0 (4)",Trevelin,-42.968260,-71.601320,"['1 cama', '1 dormitorio', '1 baño', 'Califica...",2023-07-01,2023-07-06,NaN,$45 USD,noche,1000,2023-06-24,2023-05-01,239.919355,CHUBUT
127,21925703,Hermoso Duplex con Patio y Parrilla. (4 pax).,entire_home,"4,81 (74)",Puerto Madryn,-42.787700,-65.023850,"['3 camas', '2 dormitorios', '1 baño', '1 cama...",2023-07-01,2023-07-06,NaN,$57 USD,noche,1000,2023-06-24,2023-05-01,239.919355,CHUBUT
128,644043911246503372,"Los Molles, casa familiar en entorno natural.",entire_home,NaN,Puerto Madryn,-42.735607,-65.100117,"['3 camas', '2 dormitorios', '1 baño']",2023-07-01,2023-07-07,NaN,$28 USD,noche,1000,2023-06-24,2023-05-01,239.919355,CHUBUT
129,883174783781736766,Espacio 2 - Rada Tilly,entire_home,Nuevo,Rada Tilly,-45.925810,-67.575650,"['1 cama', '1 dormitorio', '1 baño']",2023-07-01,2023-07-06,NaN,$52 USD,noche,1000,2023-06-24,2023-05-01,239.919355,CHUBUT
138,53298317,Mar de Olivillos Planta alta,entire_home,"5,0 (12)",Puerto Piramides,-42.569200,-64.273610,"['2 camas', 'Monoambiente', '1 baño', '1 cama ...",2023-07-01,2023-07-06,NaN,$75 USD,noche,1000,2023-06-24,2023-05-01,239.919355,CHUBUT
162,31081731,"Domos Puelo, para dos, con jacuzzi",entire_home,"4,98 (40)",Lago Puelo,-42.041570,-71.592370,"['1 cama', '1 dormitorio', '1 baño', '1 cama d...",2023-07-08,2023-07-15,NaN,$67 USD,noche,1000,2023-06-24,2023-05-01,239.919355,CHUBUT
182,50704005,Hostel patagónico con vista 360 de la Cordillera,shared_room,"4,86 (7)",Trevelin,-43.071450,-71.454920,"['20 camas', '4 dormitorios', '3,5 baños compa...",2023-07-01,2023-07-06,NaN,$17 USD,noche,1000,2023-06-24,2023-05-01,239.919355,CHUBUT


In [218]:
# Elimino los duplicados, que coinciden en toda la fila
df_alquileres.drop_duplicates(keep="first", inplace=True)
df_alquileres.shape

(1029, 18)

*Analisis y tratamiento de duplicados por columna*

+ Columna 1: id_alojamiento

In [219]:
# Busco si hay duplicados
df_alquileres['id_alojamiento'].duplicated(keep='first').sum()

1

In [220]:
# Identifico filas con esos valores duplicados
df_alquileres[df_alquileres['id_alojamiento'].duplicated(keep='first')]

,id_alojamiento,name,category,rating,city,latitude,longitude,characteristics,check_in,check_out,price_discounted,price_original,price_qualifier,cant_resultados,fecha_scrapping,fecha_cambio_bna,cambio_bna,provincia
648,53729814,Pequeña casa,entire_home,"5,0 (36)",San Carlos de Bariloche,-41.14768,-71.31827,"['1 cama', '1 dormitorio', '1 baño', '1 cama d...",2023-08-18,2023-08-23,NaN,$39 USD,noche,1000,2023-06-24,2023-05-01,239.919355,RIO NEGRO


In [221]:
# Muestro las filas que corresponden a los valores duplicados, todas
df_alquileres[df_alquileres['id_alojamiento'] == 53729814]

,id_alojamiento,name,category,rating,city,latitude,longitude,characteristics,check_in,check_out,price_discounted,price_original,price_qualifier,cant_resultados,fecha_scrapping,fecha_cambio_bna,cambio_bna,provincia
647,53729814,Pequeña casa,entire_home,"5,0 (36)",San Carlos de Bariloche,-41.14768,-71.31827,"['1 cama', '1 dormitorio', '1 baño', '1 cama d...",2023-08-18,2023-08-23,NaN,$39 USD,noche,1000,2023-06-24,2023-05-01,239.919355,RIO NEGRO
648,53729814,Pequeña casa,entire_home,"5,0 (36)",San Carlos de Bariloche,-41.14768,-71.31827,"['1 cama', '1 dormitorio', '1 baño', '1 cama d...",2023-08-18,2023-08-23,NaN,$39 USD,noche,1000,2023-06-24,2023-05-01,239.919355,RIO NEGRO


-> Analisis de los valores de las filas duplicadas.

In [222]:
#Verificamos los valores 
df_alquileres.loc[647]

id_alojamiento                                               53729814
name                                                     Pequeña casa
category                                                  entire_home
rating                                                       5,0 (36)
city                                          San Carlos de Bariloche
latitude                                                    -41.14768
longitude                                                   -71.31827
characteristics     ['1 cama', '1 dormitorio', '1 baño', '1 cama d...
check_in                                                   2023-08-18
check_out                                                  2023-08-23
price_discounted                                                  NaN
price_original                                                $39 USD
price_qualifier                                                 noche
cant_resultados                                                  1000
fecha_scrapping     

In [223]:
# Verificamos los valores
df_alquileres.loc[648]

id_alojamiento                                               53729814
name                                                     Pequeña casa
category                                                  entire_home
rating                                                       5,0 (36)
city                                          San Carlos de Bariloche
latitude                                                    -41.14768
longitude                                                   -71.31827
characteristics     ['1 cama', '1 dormitorio', '1 baño', '1 cama d...
check_in                                                   2023-08-18
check_out                                                  2023-08-23
price_discounted                                                  NaN
price_original                                                $39 USD
price_qualifier                                                 noche
cant_resultados                                                  1000
fecha_scrapping     

In [224]:
# Verificamos los valores para la variable que tendria fiferencia
df_alquileres.loc[647]["characteristics"]

"['1 cama', '1 dormitorio', '1 baño', '1 cama doble', 'Estacionamiento gratis', 'Wifi']"

In [225]:
# Verificamos los valores para la variable que tendria fiferencia
df_alquileres.loc[648]["characteristics"]

"['1 cama', '1 dormitorio', '1 baño', '1 cama doble', 'Estacionamiento gratis', 'Wifi', 'Calificado con 5 de 5 en limpieza']"

*ACCION:*
<BR>
*Vemos que seria el mismo registro con excepcion de la serie characteristics, donde tiene agregada comentario de calificacion*
<br>
*Como consideramos que cada valor de id_alojamiento, deberia ser unico, eliminamos el que consideramos repetido y menos se adecua a los valores esperados por serie*

In [226]:
# Descartamos el registro que no tiene agregado el comentario de la calificacion
df_alquileres.drop([648], inplace=True)
df_alquileres[df_alquileres['id_alojamiento'] == 53729814]

,id_alojamiento,name,category,rating,city,latitude,longitude,characteristics,check_in,check_out,price_discounted,price_original,price_qualifier,cant_resultados,fecha_scrapping,fecha_cambio_bna,cambio_bna,provincia
647,53729814,Pequeña casa,entire_home,"5,0 (36)",San Carlos de Bariloche,-41.14768,-71.31827,"['1 cama', '1 dormitorio', '1 baño', '1 cama d...",2023-08-18,2023-08-23,NaN,$39 USD,noche,1000,2023-06-24,2023-05-01,239.919355,RIO NEGRO


+ Columna 2: name

In [227]:
# Verifico los duplicados, que coinciden en toda la fila
df_alquileres['name'].duplicated(keep="first").sum()

1

In [228]:
# Verifico los duplicados, que coinciden en toda la fila
df_alquileres[df_alquileres['name'].duplicated(keep="first")]

,id_alojamiento,name,category,rating,city,latitude,longitude,characteristics,check_in,check_out,price_discounted,price_original,price_qualifier,cant_resultados,fecha_scrapping,fecha_cambio_bna,cambio_bna,provincia
794,800041160997364459,Vista al Lago en el Bosque - Hab Compartida,shared_room,"5,0 (8)",San Carlos de Bariloche,-41.1333,-71.36914,"['1 cama', '1 dormitorio', '1 baño compartido'...",2023-07-05,2023-07-10,NaN,$23 USD,noche,1000,2023-06-24,2023-05-01,239.919355,RIO NEGRO


In [229]:
df_alquileres[df_alquileres['name'].str.contains('Vista al Lago en el Bosque - Hab Compartida')]

,id_alojamiento,name,category,rating,city,latitude,longitude,characteristics,check_in,check_out,price_discounted,price_original,price_qualifier,cant_resultados,fecha_scrapping,fecha_cambio_bna,cambio_bna,provincia
712,797077259992479162,Vista al Lago en el Bosque - Hab Compartida,shared_room,"5,0 (8)",San Carlos de Bariloche,-41.13455,-71.36807,"['1 cama', '1 dormitorio', '1 baño compartido'...",2023-07-11,2023-07-18,NaN,$21 USD,noche,1000,2023-06-24,2023-05-01,239.919355,RIO NEGRO
794,800041160997364459,Vista al Lago en el Bosque - Hab Compartida,shared_room,"5,0 (8)",San Carlos de Bariloche,-41.13330,-71.36914,"['1 cama', '1 dormitorio', '1 baño compartido'...",2023-07-05,2023-07-10,NaN,$23 USD,noche,1000,2023-06-24,2023-05-01,239.919355,RIO NEGRO


*ACCION:*
<br>
*En este caso vemos que los nombre coinciden pero, pueden corresponder a diferentes lugares, de hecho, estan sobre diferentes calles, segun sus coordenadas.*
<br>
*Por tal motivo los dejamos a ambos registros*

+ Columna 3: category

In [230]:
# Verifico los duplicados, que coinciden en toda la fila
df_alquileres['category'].duplicated(keep="first").sum()

1025

In [231]:
# Listo los duplicados segun category
df_alquileres[df_alquileres['category'].duplicated(keep="first")].sample(10)

,id_alojamiento,name,category,rating,city,latitude,longitude,characteristics,check_in,check_out,price_discounted,price_original,price_qualifier,cant_resultados,fecha_scrapping,fecha_cambio_bna,cambio_bna,provincia
109,905128221670422939,Habitación con Altillo para 4 Pa,private_room,Nuevo,Puerto Madryn,-42.755600,-65.037001,"['3 camas', '1 dormitorio', '1 baño compartido']",2023-07-01,2023-07-06,$14 USD,$17 USD,noche,1000,2023-06-24,2023-05-01,239.919355,CHUBUT
835,836425192956909227,Hermoso monoambiente con vista!,entire_home,"5,0 (10)",BMR,-54.815580,-68.340884,"['1 cama', '1 dormitorio', '1 baño', 'Wifi', '...",2023-07-01,2023-07-06,NaN,$40 USD,noche,1000,2023-06-24,2023-05-01,239.919355,TIERRA DEL FUEGO
502,33031097,Monoambiente 40 mts2 moderno y luminoso,entire_home,"4,74 (60)",Neuquen,-38.954570,-68.077310,"['1 cama', '1 dormitorio', '1 ,5 baños', '1 ca...",2023-12-01,2023-12-06,NaN,$36 USD,noche,298,2023-06-24,2023-05-01,239.919355,NEUQUEN
792,42899197,Cuarto cálido en casa entre 3 lagos con jardín,private_room,"4,96 (55)",San Carlos de Bariloche,-41.116350,-71.435530,"['1 cama', '1 dormitorio', '1 baño compartido'...",2023-07-26,2023-07-31,NaN,$22 USD,noche,1000,2023-06-24,2023-05-01,239.919355,RIO NEGRO
376,18600709,Tu Hogar en Neuquen,entire_home,"4,86 (35)",Neuquén,-38.941740,-68.050950,"['8 camas', '2 dormitorios', '1 baño', '1 cama...",2023-07-02,2023-07-07,$135 USD,$150 USD,noche,298,2023-06-24,2023-05-01,239.919355,NEUQUEN
891,22843684,Habitación privada. Sientete como en tu casa!,private_room,"4,89 (113)",Ushuaia,-54.827000,-68.336350,"['3 camas', '2 dormitorios', '1 baño compartid...",2023-07-01,2023-07-06,NaN,$25 USD,noche,1000,2023-06-24,2023-05-01,239.919355,TIERRA DEL FUEGO
267,826285453817276970,Moderno depto con vista al mar,entire_home,NaN,Comodoro Rivadavia,-45.864460,-67.490580,"['2 camas', '1 dormitorio', '1 baño', '1 cama ...",2023-07-01,2023-07-06,NaN,$79 USD,noche,1000,2023-06-24,2023-05-01,239.919355,CHUBUT
380,872525679703009065,Depto céntrico para 5 personas,entire_home,Nuevo,Neuquén,-38.953880,-68.065820,"['4 camas', '2 dormitorios', '1 baño', '1 cama...",2023-07-24,2023-07-29,$73 USD,$91 USD,noche,298,2023-06-24,2023-05-01,239.919355,NEUQUEN
712,797077259992479162,Vista al Lago en el Bosque - Hab Compartida,shared_room,"5,0 (8)",San Carlos de Bariloche,-41.134550,-71.368070,"['1 cama', '1 dormitorio', '1 baño compartido'...",2023-07-11,2023-07-18,NaN,$21 USD,noche,1000,2023-06-24,2023-05-01,239.919355,RIO NEGRO
130,838495226449697639,Sofía. Céntrico departamento,entire_home,"5,0 (3)",Puerto Madryn,-42.769505,-65.033705,"['3 camas', '1 dormitorio', '1 baño', 'Calific...",2023-07-01,2023-07-06,NaN,$52 USD,noche,1000,2023-06-24,2023-05-01,239.919355,CHUBUT


*ACCION:*
<br>
*Entendemos que los valores de categoria se pueden repetir para los diferentes alquileres*
<br>
*por lo cual, no realizamos ninguna accion en base a esta columna*

+ Columna 4: rating

In [232]:
# Verifico los duplicado
df_alquileres['rating'].duplicated(keep="first").sum()

564

In [233]:
# Listamos algunas filas, repetidas por rating
df_alquileres[df_alquileres['rating'].duplicated(keep="first")].sample(10)

,id_alojamiento,name,category,rating,city,latitude,longitude,characteristics,check_in,check_out,price_discounted,price_original,price_qualifier,cant_resultados,fecha_scrapping,fecha_cambio_bna,cambio_bna,provincia
1004,46859084,Moderno departamento a 3 cuadras del centro.,entire_home,"4,95 (19)",AWF,-54.807968,-68.321037,"['3 camas', '1 dormitorio', '1 baño', '1 cama ...",2023-07-01,2023-07-06,$71 USD,$100 USD,noche,1000,2023-06-24,2023-05-01,239.919355,TIERRA DEL FUEGO
1042,572589539437959758,Casa particular con habitaciones privadas,private_room,"4,67 (6)",Río Grande,-53.768360,-67.707350,"['3 camas', '2 dormitorios', '1 baño compartid...",2023-07-01,2023-07-06,NaN,$35 USD,noche,1000,2023-06-24,2023-05-01,239.919355,TIERRA DEL FUEGO
300,640550672947984146,Hermoso dpto con estacionamiento incluido!,entire_home,"5,0 (17)",Neuquén,-38.940150,-68.072996,"['2 camas', '2 dormitorios', '1 baño', '1 cama...",2023-07-01,2023-07-06,NaN,$45 USD,noche,298,2023-06-24,2023-05-01,239.919355,NEUQUEN
744,649307420913365326,casa paqueta en las coloradas,entire_home,"5,0 (7)",Las Grutas,-40.828060,-65.113830,"['4 camas', '2 dormitorios', '2 baños', '1 cam...",2023-07-01,2023-07-06,NaN,$150 USD,noche,1000,2023-06-24,2023-05-01,239.919355,RIO NEGRO
74,917310050834506168,Mira ballenas desde el piso 10,entire_home,Nuevo,Puerto Madryn,-42.763690,-65.039160,"['2 camas', '1 dormitorio', '1 baño']",2023-07-07,2023-07-12,NaN,$35 USD,noche,1000,2023-06-24,2023-05-01,239.919355,CHUBUT
919,862480988954392344,Habitación doble encantadora,private_room,NaN,Ushuaia,-54.806550,-68.331390,"['1 cama', '1 dormitorio', '2 baños compartido...",2023-07-01,2023-07-08,NaN,$23 USD,noche,1000,2023-06-24,2023-05-01,239.919355,TIERRA DEL FUEGO
549,897789125691156350,Hermosa Vista y Tranquilidad. Cercano al Centro.,entire_home,Nuevo,San Carlos de Bariloche,-41.133870,-71.285220,"['1 cama', '1 dormitorio', '1 baño', '1 cama d...",2023-09-17,2023-09-22,NaN,$50 USD,noche,1000,2023-06-24,2023-05-01,239.919355,RIO NEGRO
871,830575392643151646,La mejor vista de Ushuaia,entire_home,NaN,Ushuaia,-54.793408,-68.274452,"['1 cama', '1 dormitorio', '1 baño', '1 cama q...",2023-08-30,2023-09-04,NaN,$45 USD,noche,1000,2023-06-24,2023-05-01,239.919355,TIERRA DEL FUEGO
474,829574033573661357,"Villa María, don Bosco 1125.",entire_home,NaN,Neuquén,-38.967760,-68.064260,"['No hay evaluaciones', '1 cama', '1 dormitori...",2023-07-01,2023-07-06,NaN,$55 USD,noche,298,2023-06-24,2023-05-01,239.919355,NEUQUEN
373,815951591179105029,Luminoso Departamento en Neuquén,entire_home,NaN,Neuquén,-38.948930,-68.043060,"['No hay evaluaciones', '1 cama', '1 dormitori...",2023-07-01,2023-07-06,NaN,$99 USD,noche,298,2023-06-24,2023-05-01,239.919355,NEUQUEN


In [234]:
df_alquileres['rating'].unique()

array([nan, '5,0 (6)', '4,91 (70)', 'Nuevo', '4,96 (95)', '4,86 (7)',
       '4,82 (17)', '4,96 (75)', '4,91 (67)', '4,57 (14)', '4,6 (5)',
       '4,94 (18)', '4,95 (22)', '4,92 (64)', '4,94 (109)', '4,77 (44)',
       '4,89 (35)', '5,0 (7)', '4,66 (53)', '5,0 (34)', '4,98 (93)',
       '4,96 (51)', '4,82 (66)', '4,8 (51)', '4,83 (36)', '4,9 (20)',
       '5,0 (17)', '4,65 (49)', '4,7 (33)', '4,89 (9)', '4,97 (90)',
       '5,0 (10)', '4,79 (30)', '4,86 (74)', '4,86 (44)', '4,63 (128)',
       '5,0 (3)', '4,88 (16)', '4,81 (32)', '5,0 (9)', '4,93 (27)',
       '4,92 (13)', '4,5 (4)', '4,77 (13)', '4,67 (12)', '4,9 (176)',
       '4,95 (19)', '4,8 (15)', '5,0 (14)', '4,79 (14)', '5,0 (4)',
       '5,0 (19)', '4,67 (6)', '4,82 (137)', '5,0 (11)', '4,77 (102)',
       '4,47 (55)', '4,76 (134)', '4,74 (19)', '4,96 (27)', '4,91 (11)',
       '4,93 (15)', '4,62 (13)', '4,55 (33)', '4,4 (5)', '4,72 (25)',
       '4,58 (76)', '4,73 (11)', '4,83 (60)', '4,93 (28)', '4,7 (10)',
       '4,76 (85

*ACCION:*
<br>
*Entendemos que los valores de rating se pueden repetir para los diferentes alquileres*
<br>
*Por lo cual, dejamos los registros retpetidos segun esta columna*

+ Columna 5: city

In [235]:
# Verifico los duplicados, que coinciden en toda la fila
df_alquileres['city'].duplicated(keep="first").sum()

947

In [236]:
# Listamos repetidos, por city
df_alquileres[df_alquileres['city'].duplicated(keep="first")]

,id_alojamiento,name,category,rating,city,latitude,longitude,characteristics,check_in,check_out,price_discounted,price_original,price_qualifier,cant_resultados,fecha_scrapping,fecha_cambio_bna,cambio_bna,provincia
1,51484364,Hermoso Monoambiente con Vista al Mar y Pileta,entire_home,NaN,Puerto Madryn,-42.755775,-65.037265,"['2 camas', '1 dormitorio', '1 baño', '1 cama ...",2023-07-21,2023-07-26,$18 USD,$24 USD,noche,1000,2023-06-24,2023-05-01,239.919355,CHUBUT
2,800749301724167838,Departamento Bella Vista,entire_home,"5,0 (6)",Puerto Madryn,-42.768007,-65.033108,"['3 camas', '1 dormitorio', '1 baño', 'Calific...",2023-08-01,2023-08-08,NaN,$43 USD,noche,1000,2023-06-24,2023-05-01,239.919355,CHUBUT
3,50769917,New Lil’ & Cozy - 100mts de la playa Puerto Ma...,entire_home,"4,91 (70)",Puerto Madryn,-42.777590,-65.025890,"['1 cama', '1 dormitorio', '1 baño', '1 cama q...",2023-07-09,2023-07-14,NaN,$41 USD,noche,1000,2023-06-24,2023-05-01,239.919355,CHUBUT
4,916738743930751566,Front Beach,entire_home,Nuevo,Puerto Madryn,-42.770540,-65.030810,"['2 camas', 'Monoambiente', '1 baño']",2023-07-01,2023-07-06,NaN,$35 USD,noche,1000,2023-06-24,2023-05-01,239.919355,CHUBUT
5,27877802,Luxury top 11th floor with the best 360 views,entire_home,"4,96 (95)",Puerto Madryn,-42.768330,-65.032100,"['4 camas', '2 dormitorios', '2,5 baños', '1 c...",2023-07-06,2023-07-11,NaN,$122 USD,noche,1000,2023-06-24,2023-05-01,239.919355,CHUBUT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1075,771725397497773450,Casa Fueguina en Centro,entire_home,"4,53 (17)",Ushuaia,-54.803310,-68.302580,"['3 camas', '2 dormitorios', '1 baño', 'Wifi',...",2023-08-14,2023-08-19,$60 USD,$76 USD,noche,1000,2023-06-24,2023-05-01,239.919355,TIERRA DEL FUEGO
1076,26215568,Hospedaje UshuaiAndo,entire_home,"4,82 (50)",Ushuaia,-54.800790,-68.304230,"['3 camas', '1 dormitorio', '1 baño', '1 cama ...",2023-07-23,2023-07-28,$45 USD,$50 USD,noche,1000,2023-06-24,2023-05-01,239.919355,TIERRA DEL FUEGO
1077,42486711,Yeska C Dpto. Supercéntrico. Casco Histórico,entire_home,"4,88 (8)",Ushuaia,-54.805430,-68.308710,"['1 cama', '1 dormitorio', '1 baño', '1 cama d...",2023-07-01,2023-07-06,NaN,$63 USD,noche,1000,2023-06-24,2023-05-01,239.919355,TIERRA DEL FUEGO
1078,51921207,KREN I,entire_home,"4,85 (26)",Ushuaia,-54.806170,-68.317800,"['2 camas', 'Monoambiente', '1 baño', 'Wifi', ...",2023-07-10,2023-07-15,NaN,$55 USD,noche,1000,2023-06-24,2023-05-01,239.919355,TIERRA DEL FUEGO


*ACCION:*
<br>
*Entendemos que la columna city, puede tener valores iguales para las diferentes observaciones*
<br>
*Por lo cual, no eliminamos segun los repetidos basados en esta columna*

+ Columna 6: latitude

In [237]:
# Verificamos los duplicados
df_alquileres['latitude'].duplicated(keep="first").sum()

36

In [238]:
# Listamos algunos de los duplicados
df_alquileres[df_alquileres['latitude'].duplicated(keep="first")].sample(10)

,id_alojamiento,name,category,rating,city,latitude,longitude,characteristics,check_in,check_out,price_discounted,price_original,price_qualifier,cant_resultados,fecha_scrapping,fecha_cambio_bna,cambio_bna,provincia
941,53903647,Skyline 4,entire_home,"4,88 (8)",Ushuaia,-54.807163,-68.318520,"['1 cama', '1 dormitorio', '1 baño', 'Wifi', '...",2023-07-04,2023-07-09,$88 USD,$98 USD,noche,1000,2023-06-24,2023-05-01,239.919355,TIERRA DEL FUEGO
1024,906355891628807276,Grand House,entire_home,Nuevo,Ushuaia,-54.806780,-68.315870,"['5 camas', '2 dormitorios', '2 baños', 'Wifi'...",2023-07-03,2023-07-09,$140 USD,$175 USD,noche,1000,2023-06-24,2023-05-01,239.919355,TIERRA DEL FUEGO
516,758292279446313137,Departamento un dormitorio en Complejo con Pis...,entire_home,NaN,Neuquén,-38.949740,-68.043820,"['No hay evaluaciones', '1 cama', '1 dormitori...",2023-07-01,2023-07-06,NaN,$77 USD,noche,298,2023-06-24,2023-05-01,239.919355,NEUQUEN
1019,53445794,Departamento con excelente ubicación y vistas,entire_home,"4,91 (22)",Ushuaia,-54.808500,-68.320390,"['2 camas', '1 dormitorio', '1 baño', '1 cama ...",2023-07-06,2023-07-12,NaN,$75 USD,noche,1000,2023-06-24,2023-05-01,239.919355,TIERRA DEL FUEGO
867,40366875,H) Cama en cuarto mixto compartido,shared_room,"4,91 (23)",Ushuaia,-54.796459,-68.297798,"['1 cama', '1 dormitorio', '1,5 baños comparti...",2023-07-01,2023-07-06,NaN,$12 USD,noche,1000,2023-06-24,2023-05-01,239.919355,TIERRA DEL FUEGO
318,34539785,Dpto B° Lejos de Buenos Aires.,entire_home,"4,91 (69)",Neuquén,-38.951410,-68.114000,"['3 camas', '1 dormitorio', '1 baño', '1 cama ...",2023-07-01,2023-07-06,NaN,$47 USD,noche,298,2023-06-24,2023-05-01,239.919355,NEUQUEN
657,52904852,Monoambiente 30m2 para 3. Céntrico. Calle Mitre!,entire_home,"4,89 (61)",San Carlos de Bariloche,-41.134030,-71.298840,"['3 camas', 'Monoambiente', '1 baño', '4 camas...",2023-10-07,2023-10-12,NaN,$30 USD,noche,1000,2023-06-24,2023-05-01,239.919355,RIO NEGRO
1025,623178032761843625,"departamento céntrico, cálido y cómodo.",entire_home,"4,88 (16)",AIU,-54.803600,-68.321320,"['3 camas', '1 dormitorio', '1 baño', 'Wifi', ...",2023-07-09,2023-07-14,NaN,$33 USD,noche,1000,2023-06-24,2023-05-01,239.919355,TIERRA DEL FUEGO
443,53105728,LOFT RESIDENCIAL ALTO COMAHUE DE COMPRAS!!,entire_home,NaN,Neuquén,-38.941450,-68.063080,"['No hay evaluaciones', '2 camas', 'Monoambien...",2023-07-01,2023-07-06,NaN,$20 USD,noche,298,2023-06-24,2023-05-01,239.919355,NEUQUEN
967,35647657,Ushuaia. Apartamento/Vista Exclusiva. Zona cén...,entire_home,"4,95 (111)",Ushuaia,-54.805860,-68.317640,"['2 camas', '1 dormitorio', '1 baño', '1 cama ...",2023-07-27,2023-08-01,NaN,$90 USD,noche,1000,2023-06-24,2023-05-01,239.919355,TIERRA DEL FUEGO


In [239]:
# Vemos un ejemplo
df_alquileres[df_alquileres['latitude'] == -54.803600]

,id_alojamiento,name,category,rating,city,latitude,longitude,characteristics,check_in,check_out,price_discounted,price_original,price_qualifier,cant_resultados,fecha_scrapping,fecha_cambio_bna,cambio_bna,provincia
934,31825003,Departamento con inmejorable ubicación en Ushuaia,entire_home,"4,9 (108)",Ushuaia,-54.8036,-68.30374,"['2 camas', '1 dormitorio', '1 baño', '1 cama ...",2023-07-01,2023-07-06,NaN,$85 USD,noche,1000,2023-06-24,2023-05-01,239.919355,TIERRA DEL FUEGO
1025,623178032761843625,"departamento céntrico, cálido y cómodo.",entire_home,"4,88 (16)",AIU,-54.8036,-68.32132,"['3 camas', '1 dormitorio', '1 baño', 'Wifi', ...",2023-07-09,2023-07-14,NaN,$33 USD,noche,1000,2023-06-24,2023-05-01,239.919355,TIERRA DEL FUEGO


*ACCION:*
<br>
*Entendemos que la columna latitude, puede tener valores iguales para las diferentes observaciones*
<br>
*Por lo cual, no eliminamos segun los repetidos basados en esta columna*

+ Columna 7: longitude

In [240]:
# Verificamos los duplicados
df_alquileres['longitude'].duplicated(keep="first").sum()

20

In [241]:
# Listamos algunos de los duplicados
df_alquileres[df_alquileres['longitude'].duplicated(keep="first")].sample(10)

,id_alojamiento,name,category,rating,city,latitude,longitude,characteristics,check_in,check_out,price_discounted,price_original,price_qualifier,cant_resultados,fecha_scrapping,fecha_cambio_bna,cambio_bna,provincia
1031,711709136406452484,F) Cama en cuarto mixto compartido,shared_room,"4,67 (9)",Ushuaia,-54.796494,-68.298038,"['1 cama', '1 dormitorio', '1,5 baños comparti...",2023-07-01,2023-07-06,NaN,$12 USD,noche,1000,2023-06-24,2023-05-01,239.919355,TIERRA DEL FUEGO
1071,700712866756917764,D) Cuarto doble privado,private_room,"4,85 (13)",Ushuaia,-54.796494,-68.298038,"['2 camas', '1 dormitorio', '1,5 baños compart...",2023-07-01,2023-07-06,NaN,$26 USD,noche,1000,2023-06-24,2023-05-01,239.919355,TIERRA DEL FUEGO
1044,51921353,KREN II,entire_home,"4,77 (30)",Ushuaia,-54.807290,-68.316100,"['2 camas', 'Monoambiente', '1 baño', 'Wifi', ...",2023-07-01,2023-07-06,NaN,$50 USD,noche,1000,2023-06-24,2023-05-01,239.919355,TIERRA DEL FUEGO
1058,562143078127144674,Turquesa,entire_home,NaN,Ushuaia,-54.807163,-68.318520,"['1 cama', 'Monoambiente', '1 baño', 'Wifi', '...",2023-07-04,2023-07-11,$58 USD,$64 USD,noche,1000,2023-06-24,2023-05-01,239.919355,TIERRA DEL FUEGO
15,767503846951993933,Brisa de Mar - Puerto Madryn,entire_home,"4,95 (22)",Puerto Madryn,-42.771634,-65.029071,"['3 camas', '2 dormitorios', '2,5 baños', 'Cal...",2023-08-07,2023-08-12,$34 USD,$48 USD,noche,1000,2023-06-24,2023-05-01,239.919355,CHUBUT
976,48043650,Latitud Sur Ushuaia,entire_home,"4,95 (22)",Ushuaia,-54.810780,-68.327640,"['2 camas', 'Monoambiente', '1 baño', '1 cama ...",2023-07-01,2023-07-06,NaN,$59 USD,noche,1000,2023-06-24,2023-05-01,239.919355,TIERRA DEL FUEGO
109,905128221670422939,Habitación con Altillo para 4 Pa,private_room,Nuevo,Puerto Madryn,-42.755600,-65.037001,"['3 camas', '1 dormitorio', '1 baño compartido']",2023-07-01,2023-07-06,$14 USD,$17 USD,noche,1000,2023-06-24,2023-05-01,239.919355,CHUBUT
243,909336244153347770,Bella Vista 2,entire_home,Nuevo,Esquel,-42.917491,-71.294685,"['4 camas', '1 dormitorio', '1 baño']",2023-07-01,2023-07-06,$52 USD,$65 USD,noche,1000,2023-06-24,2023-05-01,239.919355,CHUBUT
257,53295849,Preciosa unidad de alquiler de 3 dormitorios e...,entire_home,"4,84 (25)",Puerto Madryn,-42.770730,-65.029470,"['3 camas', '2 dormitorios', '2 baños', '1 cam...",2023-07-03,2023-07-08,NaN,$114 USD,noche,1000,2023-06-24,2023-05-01,239.919355,CHUBUT
527,32223913,Mono-Leloir AMUEBLADOS COMAHUE,entire_home,NaN,Neuquén,-38.941620,-68.056090,"['2 camas', 'Monoambiente', '0 baños', '1 sofá...",2023-07-01,2023-07-06,NaN,$53 USD,noche,298,2023-06-24,2023-05-01,239.919355,NEUQUEN


In [242]:
# Vemos un ejemplo
df_alquileres[df_alquileres['longitude'] == -65.029470]

,id_alojamiento,name,category,rating,city,latitude,longitude,characteristics,check_in,check_out,price_discounted,price_original,price_qualifier,cant_resultados,fecha_scrapping,fecha_cambio_bna,cambio_bna,provincia
70,20364091,Departamento Solmar,entire_home,"4,82 (137)",Puerto Madryn,-42.76963,-65.02947,"['2 camas', '1 dormitorio', '1 baño', '1 cama ...",2023-10-15,2023-10-20,NaN,$47 USD,noche,1000,2023-06-24,2023-05-01,239.919355,CHUBUT
257,53295849,Preciosa unidad de alquiler de 3 dormitorios e...,entire_home,"4,84 (25)",Puerto Madryn,-42.77073,-65.02947,"['3 camas', '2 dormitorios', '2 baños', '1 cam...",2023-07-03,2023-07-08,NaN,$114 USD,noche,1000,2023-06-24,2023-05-01,239.919355,CHUBUT


*ACCION:*
<br>
*Entendemos que la columna longitude, puede tener valores iguales para las diferentes observaciones*
<br>
*Por lo cual, no eliminamos segun los repetidos basados en esta columna*

+ Columan 8: characteristics

In [243]:
# Verificamos los duplicados
df_alquileres['characteristics'].duplicated(keep="first").sum()

243

In [244]:
# Listamos algunos de los duplicados
df_alquileres[df_alquileres['characteristics'].duplicated(
    keep="first")].sample(10)

,id_alojamiento,name,category,rating,city,latitude,longitude,characteristics,check_in,check_out,price_discounted,price_original,price_qualifier,cant_resultados,fecha_scrapping,fecha_cambio_bna,cambio_bna,provincia
308,892894728917622833,Depto Lanin-Sta Genoveva-Nqn,entire_home,Nuevo,Neuquén,-38.945644,-68.045029,"['1 cama', '1 dormitorio', '1 baño', 'Wifi']",2023-07-01,2023-07-06,$32 USD,$40 USD,noche,298,2023-06-24,2023-05-01,239.919355,NEUQUEN
631,48806077,Casa Pajaro Carpintero,entire_home,"4,92 (37)",San Carlos de Bariloche,-41.070060,-71.488750,"['3 camas', '2 dormitorios', '1 baño', '1 cama...",2023-07-01,2023-07-06,$90 USD,$100 USD,noche,1000,2023-06-24,2023-05-01,239.919355,RIO NEGRO
706,734877502966665498,Casa Bosque,entire_home,NaN,San Martín de los Andes,-40.122360,-71.290410,"['3 camas', '2 dormitorios', '2 baños', '1 cam...",2023-07-01,2023-07-06,NaN,$110 USD,noche,1000,2023-06-24,2023-05-01,239.919355,RIO NEGRO
956,53699993,Moderno estudio con jardín,entire_home,"4,94 (54)",Ushuaia,-54.809040,-68.319680,"['1 cama', 'Monoambiente', '1 baño', 'Wifi', '...",2023-10-23,2023-10-28,NaN,$55 USD,noche,1000,2023-06-24,2023-05-01,239.919355,TIERRA DEL FUEGO
629,25286446,Casita Hillside,entire_home,"4,94 (79)",San Carlos de Bariloche,-41.046010,-71.515840,"['1 cama', '1 dormitorio', '1 baño', '1 cama q...",2023-07-03,2023-07-08,NaN,$120 USD,noche,1000,2023-06-24,2023-05-01,239.919355,RIO NEGRO
733,665101374587954158,Estudio con parque,entire_home,"4,95 (19)",San Carlos de Bariloche,-41.104800,-71.440850,"['1 cama', '1 dormitorio', '1 baño', 'Estacion...",2023-07-01,2023-07-06,NaN,$69 USD,noche,1000,2023-06-24,2023-05-01,239.919355,RIO NEGRO
990,33710998,Disfrutando Sur,entire_home,"4,95 (80)",Ushuaia,-54.814190,-68.325200,"['2 camas', 'Monoambiente', '1 baño', '2 camas...",2023-07-11,2023-07-16,NaN,$57 USD,noche,1000,2023-06-24,2023-05-01,239.919355,TIERRA DEL FUEGO
625,53320962,Luminoso Departamento en Bariloche.,entire_home,"4,97 (72)",San Carlos de Bariloche,-41.136970,-71.299100,"['1 cama', '1 dormitorio', '1 baño', '1 cama d...",2023-09-29,2023-10-04,NaN,$42 USD,noche,1000,2023-06-24,2023-05-01,239.919355,RIO NEGRO
589,49107072,Contenedor-loft,entire_home,"4,91 (55)",San Carlos de Bariloche,-41.134950,-71.325710,"['1 cama', '1 dormitorio', '1 baño', '1 cama k...",2023-10-26,2023-11-01,NaN,$50 USD,noche,1000,2023-06-24,2023-05-01,239.919355,RIO NEGRO
787,889382807675939102,Acogedor departamento de 2 amb,entire_home,Nuevo,San Carlos de Bariloche,-41.122310,-71.369530,"['1 cama', '1 dormitorio', '1 baño', 'Estacion...",2023-07-05,2023-07-10,NaN,$62 USD,noche,1000,2023-06-24,2023-05-01,239.919355,RIO NEGRO


*ACCION:*
<br>
*Entendemos que la columna characteristics, puede tener valores iguales para las diferentes observaciones*
<br>
*Por lo cual, no eliminamos segun los repetidos basados en esta columna*

+ Columna 9: check_in

In [246]:
# Verificamos la cantidad de duplicados por esta columna
df_alquileres['check_in'].duplicated(keep="first").sum()

920

*ACCION:*
<br>
*Entendemos que la columna check_in, puede tener valores iguales para las diferentes observaciones*
<br>
*Por lo cual, no eliminamos segun los repetidos basados en esta columna*

+ Columna 10: check_out

In [247]:
# Verificamos la cantidad de duplicados por esta columna
df_alquileres['check_out'].duplicated(keep="first").sum()

916

*ACCION:*
<br>
*Entendemos que la columna check_out, puede tener valores iguales para las diferentes observaciones*
<br>
*Por lo cual, no eliminamos segun los repetidos basados en esta columna*

+ Columna 11: price_discounted

In [248]:
# Verificamos la cantidad de duplicados por esta columna
df_alquileres['price_discounted'].duplicated(keep="first").sum()

938

*ACCION:*
<br>
*Entendemos que la columna price_discounted, puede tener valores iguales para las diferentes observaciones*
<br>
*Por lo cual, no eliminamos segun los repetidos basados en esta columna*

+ Columna 12: price_original

In [249]:
# Verificamos la cantidad de duplicados por esta columna
df_alquileres['price_original'].duplicated(keep="first").sum()

868

*ACCION:*
<br>
*Entendemos que la columna price_original, puede tener valores iguales para las diferentes observaciones*
<br>
*Por lo cual, no eliminamos segun los repetidos basados en esta columna*

+ Columna 13: price_qualifier

In [250]:
# Verificamos la cantidad de duplicados por esta columna
df_alquileres['price_qualifier'].duplicated(keep="first").sum()

1027

*ACCION:*
<br>
*Entendemos que la columna price_qualifier, puede tener valores iguales para las diferentes observaciones*
<br>
*Por lo cual, no eliminamos segun los repetidos basados en esta columna*